## Step 1: Importing the necessary libaries

In [ ]:
import os
import tensorflow as tf
from google.colab import files
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')


## Step 2: Loading the Dataset into TensorFlow


In [ ]:
# PATH TO THE DATASET
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Research_Project/datasets/waste_dataset_v1'

# TRAINING DATASET V1 (80% OF DATA)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

# VALIDATION DATASET V1 (20% OF DATA)
val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

class_names = train_ds.class_names
print(f"Class Names: {class_names}")


## Step 3: Visualizing the images



In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):  # IT WILL TAKE ONE BATCH
    for i in range(9):  # SHOW 9 IMAGES
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # CONVERT TO PROPER FORMAT
        plt.title(class_names[labels[i]])  # ADD LABEL
        plt.axis("off")
    break
plt.show()

## Step 4: Normalize the data


In [ ]:
train_ds = train_ds.map(lambda x, y: (x / 255.0, y)) 
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

## Step 5: Building a CNN Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # CONVOLUTION LAYER
    tf.keras.layers.MaxPooling2D(2, 2),  # POOLING LAYER

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),  # CONVERT FEATURE MAPS TO 1D
    tf.keras.layers.Dense(128, activation='relu'),  # FULLY CONNECTED LAYER
    tf.keras.layers.Dense(len(class_names), activation='softmax')  # OUTPUT LAYER (SOFTMAX FOR MULTI-CLASS)
])

# COMPILE THE MODEL
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# DISPLAY THE MODEL SUMMARY
model.summary()


## Step 6: Training the model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=9
)

model.save('/content/drive/MyDrive/<path for saving>/waste_classification_model_v1.keras')
files.download('/content/drive/MyDrive/<path for saving>/waste_classification_model_v1.keras')

## Step 7: Plotting Training Results

In [ ]:
# PLOT ACCURACY AND LOSS OVER EPOCHS
plt.figure(figsize=(12, 4))

# ACCURACY PLOT
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Accuracy over Epochs")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss over Epochs")

plt.show()


## Step 8: Test the Model


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load an image
img_path = '/content/drive/MyDrive/Colab Notebooks/Research_Project/datasets/waste_dataset/glass/glass1.jpg'
img = image.load_img(img_path, target_size=(128, 128))

# Convert image to array and normalize
img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

# Predict the class
prediction = model.predict(img_array)
predicted_class = class_names[np.argmax(prediction)]

print(f"Predicted Class: {predicted_class}")
